In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pyarrow.parquet as pq
import pandas as pd
pd.options.display.max_rows = 2000
from rich import print
import warnings
import datetime
from astropy.time import Time
import os
import astropy.units as u
from astropy.coordinates import SkyCoord
from astroquery.gaia import Gaia

In [2]:
def angularSeparation(ra1, dec1, ra2, dec2):
    d2r = np.pi/180.
    ra2deg = 1./d2r
    d1 = dec1*d2r
    d2 = dec2*d2r
    r1 = ra1*d2r
    r2 = ra2*d2r
    a = np.sin((d2-d1)/2.)**2.+np.cos(d1)*np.cos(d2)*np.sin((r2-r1)/2.)**2.
    r = 2*np.arcsin(np.sqrt(a))
    return r*ra2deg

In [3]:
Gaia.MAIN_GAIA_TABLE = "gaiadr3.gaia_source"
Gaia.ROW_LIMIT = -1

In [4]:
tbl = pd.read_csv('gfcat_visit_table_positions.csv')

In [5]:
tbl

,eclipse,obj_id,ra,dec,morphology
0,927,3715900927,249.909813,41.112522,U
1,1413,1953301413,311.565113,-4.930670,F
2,1420,1468701420,315.935391,-7.379244,F
3,1422,1479501422,343.352654,-39.793072,F
4,1534,1091701534,259.445075,59.689607,F
5,1573,1783001573,261.421238,59.258315,T
6,1580,1765801580,261.421362,59.258397,T
7,1623,1417401623,259.204219,60.039535,F
8,1640,1297001640,260.307220,58.102305,F
9,1646,1567201646,258.737610,58.857582,U


In [6]:
fn = '/Users/cm/github/gphoton_working/gPhoton/aspect/aspect.parquet'

In [8]:
for i in np.arange(len(tbl))[:]:
    eclipse = int(tbl.iloc[i]['eclipse'])
    aspect = pq.read_table(fn,filters=[('eclipse','=',eclipse)]).to_pandas()
    obstart = aspect.loc[aspect['flags']%2==0]['time'].min()
    #obsend = aspect.loc[aspect['flags']%2==0]['time'].max()
    #obsdur = obsend-obstart
    GPSSECS = 315532800 + 432000
    t = obstart + GPSSECS
    dt = datetime.datetime.fromtimestamp(t)
    tbl.loc[i,'obstart']=obstart
    tbl.loc[i,'datetime_iso']=dt.isoformat().split('.')[0]
    tbl.loc[i,'datetime_decimal']=Time(dt,format='datetime').decimalyear
    ra,dec = tbl.iloc[i]['ra'],tbl.iloc[i]['dec']
    decimal_year = tbl.iloc[i]['datetime_decimal']
    coord = SkyCoord(ra=ra, dec=dec, unit=(u.degree, u.degree), frame='icrs')
    width = u.Quantity(0.2, u.deg)
    height = u.Quantity(0.2, u.deg)
    r = Gaia.query_object_async(coordinate=coord, width=width, height=height)

    # compute the offset in years between the gaia epoch (2016) and the galex epoch
    year_offset = np.array(r['ref_epoch'])-decimal_year
    # the sky positions of gaia sources in degrees
    ra_gaia,dec_gaia = np.array(r['ra']),np.array(r['dec'])
    # the proper motions of gaia sources, converted from mas to deg per year
    ra_gaia_pm,dec_gaia_pm = np.array(r['pmra'])/60/60/1000,np.array(r['pmdec'])/60/60/1000
    # "rewind" the gaia sky positions by the proper motion rate by the difference in epochs
    ra_old = ra_gaia-ra_gaia_pm*year_offset
    dec_old = dec_gaia-dec_gaia_pm*year_offset

    # ignore nan values (usually because missing pm data)
    ix_finite = np.where(np.isfinite(ra_old) & np.isfinite(dec_old) & (np.array(r["parallax_over_error"])>10))
    ix_goodsnr = np.where(np.array(r["parallax_over_error"])>10)
    d = angularSeparation(ra,dec,ra_old[ix_finite],dec_old[ix_finite])
    ix_match = np.argmin(d)
    ix = ix_finite[0][ix_match]
    tbl.loc[i,'gaia_source_id']=r[ix]['source_id']
    tbl.loc[i,'distance_uncorrected']=1000/r[ix_goodsnr]['parallax'][0]
    tbl.loc[i,'distance']=1000/r[ix]['parallax']
    tbl.loc[i,'pmra']=r[ix]['pmra']
    tbl.loc[i,'pmdec']=r[ix]['pmdec']
    tbl.loc[i,'gaia_ra']=r[ix]['ra']
    tbl.loc[i,'gaia_dec']=r[ix]['dec']
    tbl.loc[i,'gaia_match_offset']=angularSeparation(ra,dec,r[ix]['ra'],r[ix]['dec'])
    tbl.loc[i,'gaia_match_offset_uncorrected']=angularSeparation(ra,dec,
                                                                 r[ix_goodsnr]['ra'][0],
                                                                 r[ix_goodsnr]['dec'][0])
    tbl.loc[i,'n_gaia_in_17.5']=sum(d<17.5/60/60)
    tbl.loc[i,'high_pm']='Y' if ix!=0 else 'N' # the best match changed after 'rewinding'

#print(tbl.iloc[:100])

INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]


INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]


INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]


INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]


INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]


INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]


INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]


INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]


INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]


In [9]:
tbl[:][['distance','distance_uncorrected',
           'gaia_match_offset','gaia_match_offset_uncorrected',
           'high_pm','n_gaia_in_17.5',
           'pmra','pmdec']]

,distance,distance_uncorrected,gaia_match_offset,gaia_match_offset_uncorrected,high_pm,n_gaia_in_17.5,pmra,pmdec
0,3149.924855,3149.924855,0.000668,0.000668,NaN,1.0,-15.087261,-8.080175
1,191.536168,191.536168,0.000616,0.000616,NaN,1.0,17.775261,-33.688439
2,950.886176,950.886176,0.000697,0.000697,N,1.0,6.000984,-3.747007
3,104.512884,104.512884,0.000202,0.000202,NaN,1.0,65.347887,-49.015883
4,129.151695,129.151695,0.000565,0.000565,NaN,2.0,17.298796,-14.592697
5,327.920059,327.920059,0.000738,0.000738,NaN,1.0,-11.940162,-1.652595
6,327.920059,327.920059,0.000758,0.000758,NaN,1.0,-11.940162,-1.652595
7,215.613630,215.613630,0.000694,0.000694,NaN,1.0,-1.527917,1.598047
8,454.260334,454.260334,0.000703,0.000703,NaN,1.0,-7.013862,19.175729
9,470.706311,470.706311,0.000601,0.000601,NaN,1.0,-5.209685,-3.495096


In [11]:
tbl.loc[tbl['distance']!=tbl['distance_uncorrected']]

,eclipse,obj_id,ra,dec,morphology,obstart,datetime_iso,datetime_decimal,gaia_source_id,distance_uncorrected,distance,pmra,pmdec,gaia_ra,gaia_dec,gaia_match_offset,gaia_match_offset_uncorrected,n_gaia_in_17.5,high_pm
157,6986,1412206986,338.345019,-9.615185,F,7.769112e+08,2004-08-18T20:39:43,2004.630768,2.609049e+18,26.082303,26.138795,139.095633,11.732615,338.345282,-9.614923,0.000369,0.000325,2.0,N
173,7347,1906007347,338.345027,-9.615225,F,7.790477e+08,2004-09-12T14:07:39,2004.698330,2.609049e+18,26.082303,26.138795,139.095633,11.732615,338.345282,-9.614923,0.000393,0.000365,2.0,N
179,7485,1227507485,338.345036,-9.615296,F,7.798639e+08,2004-09-22T00:50:54,2004.724140,2.609049e+18,26.082303,26.138795,139.095633,11.732615,338.345282,-9.614923,0.000445,0.000435,2.0,N
248,9005,218309005,123.110331,3.555258,U,7.888587e+08,2005-01-04T02:25:46,2005.008497,3.091497e+18,1235.150529,1048.523351,0.514304,-5.575129,123.126768,3.566724,0.020015,0.019989,0.0,N
302,9625,656109625,127.906110,19.396568,F,7.925278e+08,2005-02-15T13:37:11,2005.124842,6.627321e+17,16.597931,16.477299,-207.443465,-125.513401,127.904851,19.396297,0.001219,0.001167,4.0,N
344,9990,831409990,201.511618,27.583570,F,7.946873e+08,2005-03-12T13:28:38,2005.193319,1.449302e+18,45.392500,45.729624,-2.473842,72.848851,201.511252,27.584353,0.000847,0.000745,2.0,N
353,10051,823210051,201.511627,27.583595,F,7.950482e+08,2005-03-16T17:43:37,2005.204763,1.449302e+18,45.392500,45.729624,-2.473842,72.848851,201.511252,27.584353,0.000827,0.000736,2.0,N
503,13656,-1,24.763030,-17.949130,NaN,8.163792e+08,2005-11-18T14:59:46,2005.881164,5.140694e+18,2.719520,2.674910,3178.694258,584.061335,24.771674,-17.947683,0.008350,0.008152,2.0,N
599,15953,2274415953,201.511689,27.583673,F,8.299703e+08,2006-04-24T23:18:55,2006.312251,1.449302e+18,45.392500,45.729624,-2.473842,72.848851,201.511252,27.584353,0.000782,0.000740,2.0,N
617,16312,927616312,201.511546,27.583684,F,8.320944e+08,2006-05-19T13:20:25,2006.379605,1.449302e+18,45.392500,45.729624,-2.473842,72.848851,201.511252,27.584353,0.000717,0.000626,2.0,N


In [12]:
set(tbl['morphology'])

{'E', 'F', 'T', 'U', nan}